<a href="https://colab.research.google.com/github/choycoy/final_project/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
q_df = pd.read_csv("/content/drive/My Drive/final/questions.csv",sep=",")

In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__1.csv",sep=",")
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__2.csv",sep=",")
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__3.csv",sep=",")
df4 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__4.csv",sep=",")
df5 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__5.csv",sep=",")
df6 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__6.csv",sep=",")
df7 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__7.csv",sep=",")
df8 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__8.csv",sep=",")
df9 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__9.csv",sep=",")
df10 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__10.csv",sep=",")


df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8,df9, df10])

In [ ]:
# Convert the 'question_id' column in df1 and df2 to the same data type (e.g., int64)
df['question_id'] = df['question_id'].astype('int64')
q_df['question_id'] = q_df['question_id'].astype('int64')

# Merge the dataframes on 'question_id'
merged_df = pd.merge(df, q_df[['question_id', 'correct_answer']], on='question_id', how='left')

# Create the 'is_correct' column based on the comparison between 'user_answer' and 'correct_answer'
merged_df['is_correct'] = (merged_df['user_answer'] == merged_df['correct_answer']).astype(int)

# Now merged_df will have a new column 'is_correct' indicating if the user_answer is correct or not
print(merged_df)


In [ ]:
# Assuming you have the dataframe df containing student interactions with columns: timestamp, solving_id, question_id, user_answer, elapsed_time, user_id, and is_correct (1 for correct, 0 for incorrect).

# Sort the dataframe by timestamp to ensure the data is in chronological order
merged_df.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Calculate Correctness Rate and Interaction Count for each user
merged_df['correct_count'] = merged_df.groupby('user_id')['is_correct'].cumsum()  # Cumulative sum of correct answers for each user
print(merged_df['correct_count'])

merged_df['interaction_count'] = merged_df.groupby('user_id').cumcount() + 1  # Cumulative count of interactions for each user
print(merged_df['interaction_count'])

merged_df['correctness_rate'] = merged_df['correct_count'] / merged_df['interaction_count']  # Calculate the correctness rate for each interaction

# Calculate average, minimum, and maximum elapsed time for each user
merged_df['average_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('mean')

merged_df

In [ ]:
user_correctness = merged_df.groupby('user_id')['correctness_rate'].mean()
user_correctness

In [ ]:
user_elapsed_time = merged_df.groupby('user_id')['average_elapsed_time'].mean()
user_elapsed_time

In [ ]:
user_metrics_df = pd.DataFrame({'user_id': user_correctness.index,
                                'correctness_rate': user_correctness.values,
                                'user_elapsed_time': user_elapsed_time.values})

In [ ]:
user_metrics_df['elapsed_time_scaled'] = (user_metrics_df['user_elapsed_time'] - user_metrics_df['user_elapsed_time'].min()) / (user_metrics_df['user_elapsed_time'].max() - user_metrics_df['user_elapsed_time'].min())
user_metrics_df

In [ ]:
high_achievers_threshold = 0.8
struggling_learners_threshold = 0.4

def segment_users(row):
    if row['correctness_rate'] >= high_achievers_threshold and row['elapsed_time_scaled'] <= 0.2:
        return 'High Achievers'
    elif row['correctness_rate'] < struggling_learners_threshold and row['elapsed_time_scaled'] >= 0.8:
        return 'Struggling Learners'
    else:
        return 'Average Performers'

user_metrics_df['user_segment'] = user_metrics_df.apply(segment_users, axis=1)

In [ ]:
pip install surprise

In [ ]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(0, 1))
custom_dataset = Dataset.load_from_df(merged_df[['user_id', 'question_id', 'correctness_rate']], reader)
trainset, testset = train_test_split(custom_dataset, test_size=0.2)

In [ ]:
from surprise import NMF

# Choose the NMF algorithm
algo = NMF()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)